In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from multiprocessing import Pool, Lock
import psutil

mp_lock = Lock()
process = psutil.Process(os.getpid())

data_dir = 'ml-1m'

movies_filename = 'movies.dat'
users_filename = 'users.dat'
ratings_filename = 'ratings.dat'

movies_columns = ['MovieID', 'Title', 'Genres']
users_columns = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
ratings_columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']

def make_dataframe(data_dir, filename, columns):
    data_file = os.path.join(data_dir, filename)
    return pd.read_csv(data_file, delimiter='::', names=columns, encoding='latin-1', engine='python')

movies = make_dataframe(data_dir, movies_filename, movies_columns)
users = make_dataframe(data_dir, users_filename, users_columns)
ratings = make_dataframe(data_dir, ratings_filename, ratings_columns)
data = (users, movies, ratings)

def measure_memory():
    time.sleep(2)
    mem = process.memory_info().rss
    print('Memory usage: ', mem / 1000000, ' MB')
    time.sleep(2)

def rmse(errors):
    return np.mean(errors**2)**(1/2)

def mae(errors):
    return np.mean(np.abs(errors))

def rating_errors(test_set, model):
    errors = test_set['Rating'] - test_set.apply(model, axis=1)
    return rmse(errors), mae(errors)

def crop_ratings(arr, min_lim, max_lim):
    new_arr = np.where(arr > max_lim, max_lim, arr)
    new_arr = np.where(new_arr < min_lim, min_lim, arr)
    return new_arr

def make_indices_with_holes(movie_ids):
    """
    We want for each movie the corresponding index of the column in the matrix M, 
    just like we want for each user the corresponding index of the row in the matrix U.
    For the users this is straightforward, since the the user ID's are integers from 1 
    to the number of users and we can just subtract 1 to get all the indices. For some 
    reason, some integers are skipped in the movie ID's, so we cannot use them directly 
    as indices for the M matrix. 
    
    For example: there is no movie with ID '91', so we want the movie with ID '92' 
    to correspond to M column index of 90 (remember that the first movie ID is '1' which 
    corresponds to M column index 0)
    
    Therefore we make an indices array with holes, which has parts like 
    [0, 1, ..., 218, 0, 219, 220, 221, ...]. The IDs '91' and '221' are missing. So a movie 
    with ID '222' will then take the 221st element of this array, which will take the value
    219 for the M column index, because 2 zeros are inserted for the missing IDs.
    """
    indices_with_holes = np.array([], dtype=np.int32)
    skipped_integers = 0
    for i in np.arange(len(movie_ids)):
        if movie_ids[i] == i + 1 + skipped_integers:
            indices_with_holes = np.append(indices_with_holes, i)
        else:
            while movie_ids[i] != i + 1 + skipped_integers:
                indices_with_holes = np.append(indices_with_holes, 0)
                skipped_integers += 1
            indices_with_holes = np.append(indices_with_holes, i)
    return indices_with_holes


def make_movie_column_indices(train_set, movie_ids):
    train_set_movie_ids = train_set['MovieID'].values - 1
    indices_with_holes = make_indices_with_holes(movie_ids)
    movie_column_indices = indices_with_holes[train_set_movie_ids]
    return movie_column_indices

In [ ]:
def naive_model_fold_error(train_index, test_index, model_type, model_specific_params):
    train_set = ratings.iloc[train_index]
    test_set = ratings.iloc[test_index]

    mean_rating = train_set['Rating'].mean()

    model = make_naive_model(mean_rating, train_set, model_type, model_specific_params)

    rms_error, ma_error = rating_errors(test_set, model)
    
    mp_lock.acquire()
    print('rmse ', rms_error, ' mae ', ma_error)
    mp_lock.release()
    
    measure_memory()
    
    return rms_error, ma_error

def test_naive_model(data, model_type, folds=5):
    users, movies, ratings = data
    
    cv = KFold(n_splits=folds, random_state=42, shuffle=True)
    
    if model_type == '1':
        model_specific_params = {}
    elif model_type == '2':
        model_specific_params = {'user_indices': users['UserID']}
    elif model_type == '3':
        model_specific_params = {'movie_ids': movies['MovieID']}
        model_specific_params.update({'indices_with_holes': 
                                      make_indices_with_holes(model_specific_params['movie_ids'])})
    else:
        return ValueError(f'Model type {model_type} is unknown.')
    
    params = [(train_index, 
               test_index, 
               model_type,
               model_specific_params) for train_index, test_index in cv.split(ratings)]
    pool = Pool(5)
    fold_errors = pool.starmap(naive_model_fold_error, params)
    pool.close()
    pool.join()
    
    rms_errors = np.array([])
    ma_errors = np.array([])
    
    for errors in fold_errors:
        rms_errors = np.append(rms_errors, errors[0])
        ma_errors = np.append(ma_errors, errors[1])
        
    mean_rms_error = np.mean(rms_errors)
    mean_ma_error = np.mean(ma_errors)

    print('mean rms error', mean_rms_error)
    print('mean ma error', mean_ma_error)
    
def make_naive_model(mean_rating, train_set, model_type, model_specific_params):
    if model_type == '1':
        def model(row):
            return mean_rating
        return model
    elif model_type == '2':
        mean_rating_per_user = np.array([])
        
        for user_index in model_specific_params['user_indices']:
            subset = train_set[train_set['UserID'].values == user_index]
            mean_rating_per_user = np.append(mean_rating_per_user, np.mean(subset['Rating']))

        mean_rating_per_user = np.where(np.isnan(mean_rating_per_user), mean_rating, mean_rating_per_user)

        def model(row):
            user_id = row['UserID'] - 1
            return mean_rating_per_user[user_id]
        return model
    elif model_type == '3':
        mean_rating_per_movie = np.array([])
        
        for movie_index in model_specific_params['movie_ids']:
            subset = train_set[train_set['MovieID'].values == movie_index]
            mean_rating_per_movie = np.append(mean_rating_per_movie, np.mean(subset['Rating']))

        mean_rating_per_movie = np.where(np.isnan(mean_rating_per_movie), mean_rating, mean_rating_per_movie)

        def model(row):
            movie_id = row['MovieID'] - 1
            return mean_rating_per_movie[model_specific_params['indices_with_holes'][movie_id]]
        return model
    else:
        return ValueError(f'Model type {model_type} is unknown.')

In [ ]:
t0 = time.time()

test_naive_model(data, model_type='1')

duration = time.time() - t0 - 4
print('Run time: ', duration, ' sec')

In [ ]:
t0 = time.time()

test_naive_model(data, model_type='2')

duration = time.time() - t0 - 4
print('Run time: ', duration, ' sec')

In [ ]:
t0 = time.time()

test_naive_model(data, model_type='3')

duration = time.time() - t0 - 4
print('Run time: ', duration, ' sec')

In [ ]:
# Overall mean rating 
mean_rating = ratings['Rating'].mean()

# Lookup tables for naive models 2, 3, 4 and 5
mean_rating_per_user = {user_id : ratings[ratings['UserID'] == user_id]['Rating'].mean() for user_id in users['UserID']}
mean_rating_per_movie = {movie_id : ratings[ratings['MovieID'] == movie_id]['Rating'].mean() for movie_id in movies['MovieID']}

def test_error_4_5(test_set,reg):
    
    mean_rating_per_movie_list = np.array([mean_rating_per_movie[movie_id] for movie_id in test_set['MovieID']])
    mean_rating_per_user_list = np.array([mean_rating_per_user[user_id] for user_id in test_set['UserID']])
    
    ##the predicted rating value for the test set using model 4 and 5.
    mean_rating_list = np.vstack((mean_rating_per_movie_list,mean_rating_per_user_list)).T
    pre_rating = reg.predict(mean_rating_list)[0]
    
    rating_error = ((test_set['Rating'] - pre_rating)**2).mean()**(1/2)
    
    return rating_error

In [ ]:
def test_naive_model_4_5(data, subset:int=None):
    users, movies, ratings = data
    
    cv = KFold(n_splits=5, random_state=42, shuffle=True)
    
    rating_errors_4 = np.array([])
    rating_errors_5 = np.array([])
    
    for train_index, test_index in cv.split(ratings):
        train_set = ratings.iloc[train_index]
        test_set = ratings.iloc[test_index]
        user_ids = train_set['UserID']
        
        mean_rating_per_movie = {movie_id : train_set[train_set['MovieID'] == movie_id]['Rating'].mean() for movie_id in movies['MovieID']}
        mean_rating_per_user = {user_id : train_set[train_set['UserID'] == user_id]['Rating'].mean() for user_id in users['UserID']}
        
        
        ##the lists of mean Ritem and mean Ruser for each rating in the train_set
        mean_rating_per_movie_list = np.array([mean_rating_per_movie[movie_id] for movie_id in train_set['MovieID']])
        mean_rating_per_user_list = np.array([mean_rating_per_user[user_id] for user_id in train_set['UserID']])
        
        ## stack the Ritem and Ruser lists for linear fitting
        mean_rating_list = np.vstack((mean_rating_per_movie_list, mean_rating_per_user_list)).T
        
        ## uisng Ordinary least squares Linear Regression to find alpha beta and gamma
        reg_4 = LinearRegression(fit_intercept=False).fit(mean_rating_list, train_set['Rating'])
        reg_5 = LinearRegression(fit_intercept=True).fit(mean_rating_list, train_set['Rating'])

        
        rating_err_4 = test_error_4_5(test_set,reg_4)
        rating_err_5 = test_error_4_5(test_set,reg_5)
        
        rating_errors_4 = np.append(rating_errors_4,rating_err_4)
        rating_errors_5 = np.append(rating_errors_5,rating_err_5)
        
        print('Rating Error for Naive Model 4:', rating_err_4)
        print('Rating Error for Naive Model 5:', rating_err_5)
    
    print('Mean Rating Error for Naive Model 4:', np.mean(rating_errors_4))   
    print('Mean Rating Error for Naive Model 5:', np.mean(rating_errors_5))

In [ ]:
test_naive_model_4_5(data)

In [ ]:
def mf_fold_error(train_index, test_index, user, movies, factors, n_training_steps, lr, lam):
    n_users = len(users)
    n_movies = len(movies)
    
    U = np.random.normal(0.0, 1.0, ((n_users, factors)))
    M = np.random.normal(0.0, 1.0, ((factors, n_movies)))
    
    train_set = ratings.iloc[train_index]
    test_set = ratings.iloc[test_index]

    train_user_indices = train_set['UserID'].values - 1
    test_user_indices = test_set['UserID'].values - 1

    movie_ids = movies['MovieID'].values
    train_movie_indices = make_movie_column_indices(train_set, movie_ids)
    test_movie_indices = make_movie_column_indices(test_set, movie_ids)
    
    train_set_rating_matrix = np.zeros((n_users, n_movies))
    for u, m, rating in zip(train_user_indices, train_movie_indices, train_set['Rating']):
        train_set_rating_matrix[u, m] = rating

    train_rmses = np.array([])
    test_rmses = np.array([])

    train_maes = np.array([])
    test_maes = np.array([])

    for step in range(n_training_steps):
        predicted_ratings = np.dot(U, M)

        train_set_predicted_ratings = predicted_ratings[train_user_indices, train_movie_indices]
        test_set_predicted_ratings = predicted_ratings[test_user_indices, test_movie_indices]

        train_errors = train_set['Rating'] - crop_ratings(train_set_predicted_ratings, 1.0, 5.0)
        test_errors = test_set['Rating'] - crop_ratings(test_set_predicted_ratings, 1.0, 5.0)
        
        train_rmse, test_rmse = rmse(train_errors), rmse(test_errors)
        
        train_rmses = np.append(train_rmses, train_rmse)
        test_rmses = np.append(test_rmses, test_rmse)

        train_maes = np.append(train_maes, mae(train_errors))
        test_maes = np.append(test_maes, mae(test_errors))

        mp_lock.acquire()
        print('step ', step, ' rmse train ', train_rmse, ' rmse test ', test_rmse)
        mp_lock.release()
        
        for u, m in zip(train_user_indices, train_movie_indices):
            error = train_set_rating_matrix[u, m] - np.dot(U[u, :], M[:, m])
            U[u, :] += lr * (2 * error * M[:, m] - lam * U[u, :])
            M[:, m] += lr * (2 * error * U[u, :] - lam * M[:, m])
        
        if step == n_training_steps - 1:
            measure_memory()
    
    return train_rmses, test_rmses, train_maes, test_maes


def matrix_factorization(data, factors, lr, n_training_steps, lam, n_folds):
    users, movies, ratings = data

    rating_errors = np.array([])

    cv = KFold(n_splits=n_folds, random_state=42, shuffle=True)

    print('Initialize...')

    params = [(train_index, 
               test_index, 
               users, 
               movies, 
               factors, 
               n_training_steps, 
               lr, 
               lam) for train_index, test_index in cv.split(ratings)]
    pool = Pool(5)
    fold_errors = pool.starmap(mf_fold_error, params)
    pool.close()
    pool.join()
    
    train_rmses_per_fold = np.array([])
    test_rmses_per_fold = np.array([])

    train_maes_per_fold = np.array([])
    test_maes_per_fold = np.array([])
    
    for errors in fold_errors:
        train_rmses, test_rmses, train_maes, test_maes = errors
        train_rmses_per_fold = np.append(train_rmses_per_fold, train_rmses)
        test_rmses_per_fold = np.append(test_rmses_per_fold, test_rmses)
        train_maes_per_fold = np.append(train_maes_per_fold, train_maes)
        test_maes_per_fold = np.append(test_maes_per_fold, test_maes)
    
    return np.stack((train_rmses_per_fold, 
                     test_rmses_per_fold, 
                     train_maes_per_fold, 
                     test_maes_per_fold))

number_of_folds = 5
number_of_training_steps = 75
number_of_factors = 10
learning_rate = 0.005
regularization = 0.05

np.random.seed(42)

t0 = time.time()

errors = matrix_factorization(data,
                              factors=number_of_factors,
                              lr=learning_rate,
                              n_training_steps=number_of_training_steps,
                              lam=regularization,
                              n_folds=number_of_folds)

duration = time.time() - t0 - 4

print('Run time: ', duration, ' sec')

In [ ]:
mean_train_rmses = np.mean(np.reshape(errors[0], (number_of_folds, number_of_training_steps)), axis=0)
mean_test_rmses = np.mean(np.reshape(errors[1], (number_of_folds, number_of_training_steps)), axis=0)
mean_train_maes = np.mean(np.reshape(errors[2], (number_of_folds, number_of_training_steps)), axis=0)
mean_test_maes = np.mean(np.reshape(errors[3], (number_of_folds, number_of_training_steps)), axis=0)

training_steps = np.arange(number_of_training_steps)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.plot(training_steps, mean_train_rmses, label='train rmse')
ax.plot(training_steps, mean_test_rmses, label='test rmse')
ax.plot(training_steps, mean_train_maes, label='train mae')
ax.plot(training_steps, mean_test_maes, label='test mae')
ax.set_title('Matrix factorization errors')
ax.set_xlabel('training step')
ax.set_ylabel('error')
ax.legend()
plt.show()